In [1]:
import glob
import pandas as pd
import numpy as np
import pickle
import re
from bs4 import BeautifulSoup
import collections
import requests
from itertools import chain
from tqdm import tqdm

In [3]:
# Load POWO
powo = pickle.load(open('../data/processed/description_web_powo.pkl', 'rb'))
# Load all powo data
ipni_data = pd.read_csv('../data/external/ipni.csv', header=None)
ipni_data.columns = ['link', 'species', 'subname', 
                     'x4',    'x5',     'x6',
                     'x7',    'x8',     'x9', 
                     'x10',   'x11',    'x12',]

# List species
powo_list = list(powo.keys())
# Drop species without description
inpi_df = ipni_data[ipni_data['species'].isin(powo_list)]

In [5]:
#ipni_data = inpi_df.iloc[0:20, :]

# loop over URLs
for ipni_link, species, subname in zip(tqdm(inpi_df.link), inpi_df.species, inpi_df.subname):
    
    #print(species)
    
    URL = 'http://powo.science.kew.org/taxon/' + ipni_link
    #print(URL)
    
    try:
        page = requests.get(URL, timeout=5)
        soup = BeautifulSoup(page.content, 'html.parser')
        # Create folder
        # Dump the HTML file
        with open('../data/raw/POWO/' + species + ' - ' + subname + '.html', "w") as f:
              f.write(str(soup))
    except:
        continue

100%|███████████████████████████████████| 41713/41713 [5:00:32<00:00,  2.31it/s]


In [108]:
# Read files
html_list = glob.glob('../data/raw/POWO/*')
# Open dict
data_powo = collections.defaultdict(list)

for html in tqdm(html_list):
    # Open HTML file
    with open(html) as f:
        soup = BeautifulSoup(f, 'html.parser')

        # Extract title
        name = html.lstrip('../data/raw/POWO/').split(' - ')[0]
        
        # Loop over text
        for text in soup.find_all('dd'):
            try:
                if text.find_previous_sibling().span.text == 'Morphology':
                    data_powo[name].append(text.text.strip())
            except:
                continue
    
with open('../data/processed/train_dataPOWO.pkl', 'wb') as f:
    pickle.dump(data_powo, f)    

100%|█████████████████████████████████████| 40187/40187 [15:10<00:00, 44.15it/s]
